In [2]:
import math
import multiprocessing as mg
import multiprocessing.pool
# import pys2let as ps
import random
import string
import itertools
import os

import jax
jax.config.update("jax_enable_x64", True)
import s2fft
import healpy as hp
import numpy as np
import s2wav
import s2wav
import matplotlib.pyplot as plt
%matplotlib inline 
import skyclean
from skyclean import CMB_data

In [15]:
def mw_alm_2_hp_alm(MW_alm, lmax):
    '''MW_alm: 2D array of shape (Lmax, 2*Lmax-1) (MW sampling, McEwen & Wiaux)
    '''
    # Initialize the 1D hp_alm array with the appropriate size
    hp_alm = np.zeros(hp.Alm.getsize(lmax), dtype=np.complex128)
        
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            index = hp.Alm.getidx(lmax, l, abs(m))
            if m < 0:
                hp_alm[index] = (-1)**m * np.conj(MW_alm[l, lmax + m])
            else:
                hp_alm[index] = MW_alm[l, lmax + m]

    return hp_alm


def Single_Map_doubleworker(MW_Pix_Map):
    '''
    Input: MW_Pix_Map: list of mw maps at different scales 
    Each pixel map is a wavelet pixel map of shape (1, Lmax, 2*Lmax-1) (MW sampling, McEwen & Wiaux)
    It is the output of s2wav.analysis
    (Scale: 0, size (1, 4, 7))

    Process:
    1. Covert MW Pixel Map to MW alm space using s2fft.forward

    2. Add zero to the mw alms  (Is it correct? or should I add zeros to the hp alm's and then convert to mw alm's)
    by adding zeros to the MW alm's we are increasing the resolution of the map
    Double the rows of the mw alms, since, the number of rows represents the L (level of detail)
    
    3. Convert mw alm to mw map 
    
    '''
    # print("original Pixel Map size", MW_Pix_Map.shape)
    MW_alm = s2fft.forward(MW_Pix_Map, L = MW_Pix_Map.shape[1])
    # print("original alm size", MW_alm.shape)
    
   
    # print("Scale:",i,"original alm size", MW_alm[i].shape)
    padded_alm = np.zeros((MW_alm.shape[0]*2,MW_alm.shape[1]*2))
    # stored_wavelet_coeffs_alm_doubled.append(skyclean.double_resolution(stored_wavelet_coeffs_alm[i]))
    padded_alm[:MW_alm.shape[0], :MW_alm.shape[1]] = MW_alm
    # print("padded alm size", padded_alm.shape)
    MW_alm_doubled = padded_alm
    
    MW_Pix_Map_doubled = s2fft.inverse(MW_alm_doubled, L = MW_alm_doubled.shape[0])
    # print("Scale:","doubled map size", MW_Pix_Map_doubled.shape)

    return MW_Pix_Map_doubled


def smoothed_covariance(MW_Map1, MW_Map2):
    '''
    Input: MW_Map1, MW_Map2: same size MW pixel wavelet maps at different frequencies
    output: R_map: smoothed covariance map beteen MW_Map1 and MW_Map2
    '''
    smoothing_lmax = MW_Map1.shape[0]
    # Get the real part of the map
    map1 = np.real(MW_Map1)
    map2 = np.real(MW_Map2)
    # Covariance matrix
    R_MW_Pixel_map = np.multiply(map1,map2) + 0.j #Add back in zero imaginary part
    # print("R", R_MW_Pixel_map.shape)

    # smoothing in harmonic space for efficiency
    R_MW_alm = s2fft.forward(R_MW_Pixel_map, L = smoothing_lmax)
    # print("R_MW_alm", R_MW_alm.shape)


    nsamp = 1200.0
    lmax_at_scale_j = R_MW_alm.shape[0]
    npix = hp.nside2npix(1<<(int(0.5*lmax_at_scale_j)-1).bit_length())
    # (int(0.5*scale_lmax)-1).bit_length() calculates the number of bits necessary to represent the integer int(0.5*scale_lmax)-1 in binary.
    # 1 << (int(0.5*scale_lmax)-1).bit_length() performs a bitwise left shift, essentially calculating 2^(number of bits).
    scale_fwhm = 4.0 * math.sqrt(nsamp / npix)
    # for high resolution maps, it is still the same number pixels sampled by the actual range is smaller.
    # the beam will become very narrow.


    gauss_smooth = hp.gauss_beam(scale_fwhm,lmax=smoothing_lmax-1)
    MW_alm_beam_convolved = np.zeros(R_MW_alm.shape, dtype=np.complex128)

    # Convolve the MW alms with the beam
    for i in range(R_MW_alm.shape[1]):
        MW_alm_beam_convolved[:, i] = R_MW_alm[:, i] * gauss_smooth
    
    R_covariance_map = s2fft.inverse(MW_alm_beam_convolved, L = smoothing_lmax)

    return R_covariance_map




 

## Loaded mw wavelet coefficient map
stored_wavelet_coeffs_pix = [np.load(f"../convolution/wavelet_coefficient/wav_30_{i}.npy", allow_pickle=True) for i in range(12)]
stored_scaling_coeffs_pix = np.load("../convolution/scaling_coefficient/scal_30.npy")


# print(stored_wavelet_coeffs_pix[0].shape)
stored_wavelet_coeffs_pix = stored_wavelet_coeffs_pix[:3]

wavelet_MW_Pix_Map_doubled = Single_Map_doubleworker(stored_wavelet_coeffs_pix[0])

# display(wavelet_MW_Pix_Map_doubled[0])
# Why oen dimension is reduced?
# Is it the spin?
print(wavelet_MW_Pix_Map_doubled.shape)


R_covariance_map = smoothed_covariance(wavelet_MW_Pix_Map_doubled, wavelet_MW_Pix_Map_doubled)

R_covariance_map.shape

(8, 15)


(8, 15)

In [31]:
# we don't want to store all the data in a dictionary 
# wavelet is wavelet, and alm is alm 
# we get them from the same function

import numpy as np

def load_frequency_data(base_path, file_template, frequencies, scales=None):
    """
    Load NumPy arrays from dynamically generated file paths for each frequency and scale.
    
    Args:
        base_path (str): The base path where the files are located.
        file_template (str): The template for the file names, with placeholders for frequency and scale.
        frequencies (list): A list of frequency names.
        scales_: A lists of scales.
        
    Returns:
        dict: A dictionary where keys are tuples of (frequency, scale) and values are loaded NumPy arrays.
    """
    frequency_data = {}
    for frequency in frequencies:
        for scale in scales:
            # Generate the file path using the template and the current frequency and scale
            path = f"{base_path}/{file_template.format(frequency, scale)}"
            try:
                frequency_data[(frequency, scale)] = np.load(path, allow_pickle=True)
            except Exception as e:
                print(f"Error loading {path} for frequency {frequency} and scale {scale}: {e}")
    return frequency_data



base_path = "../convolution/wavelet_coefficient"
file_template = "wav_{}_{}.npy"
frequencies = ['030', '070']
scales = [0, 1, 2]

original_wavelet_c_j = load_frequency_data(base_path, file_template, frequencies, scales)

# for (frequency, scale), data in frequency_data.items():
#     print(f"Frequency: {frequency}, Scale: {scale}, Data shape: {data.shape}")




In [32]:
for i in frequencies:
    for j in scales:
        # print(frequency_data[(i,j)].shape)
        wavelet_MW_Pix_Map_doubled = Single_Map_doubleworker(original_wavelet_c_j[(i,j)])
        np.save(f"../convolution/wavelet_coefficient_doubled/wav_{i}_{j}.npy", wavelet_MW_Pix_Map_doubled)

In [48]:
doubled_MW_wav_c_j = load_frequency_data("../convolution/wavelet_coefficient_doubled", "wav_{}_{}.npy", frequencies, scales)
print(doubled_MW_wav_c_j.keys())

total_frequency = len(frequencies)
convariance_matrix_all_freq = np.full((len(scales),total_frequency, total_frequency), None)
# Go to each scale in each frequency
# for i in frequencies:
for j in scales:
    # calculate the covariance matrix
    for i in range(total_frequency):
        for fq in range(i, total_frequency):
            # print(f"Element at ({i}, {fq}): {convariance_matrix_all_freq[j, i, fq]}")
            convariance_matrix_all_freq[j, i, fq] = smoothed_covariance(doubled_MW_wav_c_j[(frequencies[i],j)], doubled_MW_wav_c_j[(frequencies[fq],j)])
            np.save(f"../convolution/covariance_matrix/cov_{j}_{frequencies[i]}_{frequencies[fq]}.npy", convariance_matrix_all_freq[j, i, fq])
            print(f"Element at (scale {j}, covariance of frequency {frequencies[i]} and frqeucny {frequencies[fq]}): {convariance_matrix_all_freq[j, i, fq].shape}")
            
    # Symmetric 
    # The outer loop starts from 1 (range(1, total_frequency)) to skip the diagonal element in the first row.
    # The inner loop runs from 0 to i-1 (range(i)) to exclude the diagonal and include only the lower triangular elements.

    for l1 in range(1, total_frequency):
        for l2 in range(l1):
            convariance_matrix_all_freq[j, l1, l2] = convariance_matrix_all_freq[j, l2, l1]

            # print(f"Element at ({i}, {j}): {matrix[i, j]}")
# symmetric_matrix = np.tril(convariance_matrix_all_freq[0, 1, 1]) + np.tril(convariance_matrix_all_freq[0, 1, 1], -1).T
# print(symmetric_matrix.shape)
print(convariance_matrix_all_freq.shape)
# convariance_matrix_all_freq[2, 0, 1]

dict_keys([('030', 0), ('030', 1), ('030', 2), ('070', 0), ('070', 1), ('070', 2)])
Element at (scale 0, covariance of frequency 030 and frqeucny 030): (8, 15)
Element at (scale 0, covariance of frequency 030 and frqeucny 070): (8, 15)
Element at (scale 0, covariance of frequency 070 and frqeucny 070): (8, 15)
Element at (scale 1, covariance of frequency 030 and frqeucny 030): (8, 15)
Element at (scale 1, covariance of frequency 030 and frqeucny 070): (8, 15)
Element at (scale 1, covariance of frequency 070 and frqeucny 070): (8, 15)
Element at (scale 2, covariance of frequency 030 and frqeucny 030): (16, 31)
Element at (scale 2, covariance of frequency 030 and frqeucny 070): (16, 31)
Element at (scale 2, covariance of frequency 070 and frqeucny 070): (16, 31)
(3, 2, 2)


None


In [6]:
# total_frequency = 2  # Size of the matrix
# convariance_matrix_all_freq = np.full((total_frequency, total_frequency), None)

# for i in range(total_frequency):
#     for j in range(i, total_frequency):
#         print(f"Element at ({i}, {j}): {convariance_matrix_all_freq[i, j]}")
#         convariance_matrix_all_freq[i, j] = smoothed_covariance(stored_wavelet_coeffs_pix[i], stored_wavelet_coeffs_pix[j])

array([[None, None],
       [None, None]], dtype=object)

In [58]:
print(convariance_matrix_all_freq[0,0,0][0,0])
print(convariance_matrix_all_freq[0,0,1][0,0])
print(convariance_matrix_all_freq[0,1,0][0,0])
print(convariance_matrix_all_freq[0,1,1][0,0])

(2.6790238831079387e-41+1.6601288232721494e-59j)
(-2.0792968206583024e-42-5.396562979065225e-59j)
(-2.0792968206583024e-42-5.396562979065225e-59j)
(5.991679955824472e-43+2.6452465717748307e-60j)


In [ ]:
convariance_matrix_all_freq[1,"070","030"][0,0]

In [49]:
convariance_matrix_all_freq

array([[[array([[2.67902388e-41+1.66012882e-59j, 2.67902388e-41+1.66012907e-59j,
                 2.67902388e-41+1.66012907e-59j, 2.67902388e-41+1.66012912e-59j,
                 2.67902388e-41+1.66012920e-59j, 2.67902388e-41+1.66012886e-59j,
                 2.67902388e-41+1.66012870e-59j, 2.67902388e-41+1.66012849e-59j,
                 2.67902388e-41+1.66012828e-59j, 2.67902388e-41+1.66012814e-59j,
                 2.67902388e-41+1.66012794e-59j, 2.67902388e-41+1.66012814e-59j,
                 2.67902388e-41+1.66012822e-59j, 2.67902388e-41+1.66012829e-59j,
                 2.67902388e-41+1.66012863e-59j],
                [2.67902389e-41+1.66012905e-59j, 2.67902389e-41+1.66012960e-59j,
                 2.67902389e-41+1.66012984e-59j, 2.67902388e-41+1.66012951e-59j,
                 2.67902388e-41+1.66012923e-59j, 2.67902388e-41+1.66012908e-59j,
                 2.67902388e-41+1.66012846e-59j, 2.67902388e-41+1.66012773e-59j,
                 2.67902388e-41+1.66012740e-59j, 2.67902388

In [52]:
convariance_matrix_R_each_P_all_freq = np.full((len(scales),total_frequency, total_frequency), None)

for j in range(len(scales)):
    for i in range(total_frequency):
        for fq in range(total_frequency):
            for row in range(convariance_matrix_all_freq[j, i, fq].shape[0]):
                for col in range(convariance_matrix_all_freq[j, i, fq].shape[1]):
                    convariance_matrix_R_each_P_all_freq[j, i, fq] = convariance_matrix_all_freq[j, i, fq][row,col]
                    break
print(convariance_matrix_R_each_P_all_freq)
            
            
print(convariance_matrix_R_each_P_all_freq.shape)

(2.6790238831079387e-41+1.6601288232721494e-59j)
(-2.0792968206583024e-42-5.396562979065225e-59j)
(-2.0792968206583024e-42-5.396562979065225e-59j)
(5.991679955824472e-43+2.6452465717748307e-60j)

[[[(2.679023890978598e-41+1.6601264151471852e-59j)
   (-2.0792968232651058e-42-5.396563157264586e-59j)]
  [(-2.0792968232651058e-42-5.396563157264586e-59j)
   (5.991679930130785e-43+2.645246706345565e-60j)]]

 [[(4.5161683639923993e-11+6.267921834760875e-28j)
   (-1.303826161332167e-12+1.3257326138014007e-28j)]
  [(-1.303826161332167e-12+1.3257326138014007e-28j)
   (1.278591100390293e-11+1.2727256301828028e-28j)]]

 [[(2.0254279127039575e-10+2.4905114904572514e-27j)
   (-8.552826800317619e-12+4.54557903119268e-28j)]
  [(-8.552826800317619e-12+4.54557903119268e-28j)
   (1.9403315355233835e-11-4.1014457982992075e-29j)]]]
(3, 2, 2)


In [ ]:
# Example 4x4 matrix
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
]

n = len(matrix)  # Size of the matrix

# Loop through the upper triangular part of the matrix
for i in range(n):
    for j in range(i, n):
        print(f"Element at ({i}, {j}): {matrix[i][j]}")

In [ ]:
# a = np.array([[1,2],[3,4]])
# b = np.array([[5,6],[7,8]])
# display(a)
# display(b)
# display(np.multiply(a,b))
# display(np.multiply(wavelet_MW_Pix_Map_doubled[0],wavelet_MW_Pix_Map_doubled[1]))

In [ ]:
def ILC(MW_Pix_Map):

    
    Current_Wavelet_Map = MW_Pix_Map
    
    # Define the size of the smoothing beam 
    # 1200 pixels, size of the sphere
    nsamp = 1200.0
    lmax_at_scale_j = Current_Wavelet_Map.shape[0]
    npix = hp.nside2npix(1<<(int(0.5*lmax_at_scale_j)-1).bit_length())
    # (int(0.5*scale_lmax)-1).bit_length() calculates the number of bits necessary to represent the integer int(0.5*scale_lmax)-1 in binary.
    # 1 << (int(0.5*scale_lmax)-1).bit_length() performs a bitwise left shift, essentially calculating 2^(number of bits).
    scale_fwhm = 4.0 * math.sqrt(nsamp / npix)
    # for high resolution maps, it is still the same number pixels sampled by the actual range is smaller.
    # the beam will become very narrow.
    print(lmax_at_scale_j,npix, scale_fwhm)


    # after smoothworker, 81 covariance maps 


    

# ILC(wavelet_MW_Pix_Map_doubled)

# print(wavelet_MW_Pix_Map_doubled.shape)
    

